### BERT FINETUNING on Custom DataSet

In [1]:
import numpy as np,pandas as pd

In [9]:
import requests
import json
import torch
import os
from tqdm import tqdm

In [149]:
import os

# Define the directory path
directory_path = './BERT-SQuAD'  # Use relative path for Kaggle

# Check if the directory exists, if not, create it
if not os.path.exists(directory_path):
    os.mkdir(directory_path)

print(f"Directory '{directory_path}' created.")

Directory './BERT-SQuAD' created.


In [4]:
!pip install transformers

# Squad Dataset Download

In [343]:
# Load the training dataset and take a look at it
with open('/kaggle/input/rag-data/RAG_Datasets/BertTrainableDataset.json', 'rb') as f:
  squad = json.load(f)

In [344]:
squad['data'][0].keys()

dict_keys(['title', 'paragraphs'])

In [345]:
from pprint import pprint

In [346]:
pprint(squad['data'][0]["title"])

'Cyber Security Overview'


In [347]:
len(squad["data"])

1

In [348]:
gr = -1
for idx, group in enumerate(squad['data']):
  print(group['title'])
  if group['title'] == 'Cyber Security Overview':
    gr = idx
    print(gr)
    break

Cyber Security Overview
0


In [349]:
print(len(squad["data"][0]["paragraphs"]))

50


In [350]:
# and this is the context given for NYC
squad['data'][0]['paragraphs'][49]['context']

'The company also announced innovations like Singularity Hyperautomation, a no-code automation solution designed to streamline SOC workflows and reduce response times, particularly for complex threats such as ransomware. This solution integrates into the security analyst workflow, automatically generating playbooks based on peer-driven insights, making responses faster and more efficient. Another notable development is the Purple AI-powered Alert AutoTriage feature, which uses AI to assess and prioritize alerts, helping teams focus on real threats and reduce alert fatigue.'

In [351]:
def read_data(path):  
  # load the json file
  with open(path, 'rb') as f:
    squad = json.load(f)

  contexts = []
  questions = []
  answers = []

  for group in squad['data']:
    for passage in group['paragraphs']:
      context = passage['context']
      for qa in passage['qas']:
        question = qa['question']
        for answer in qa['answers']:
          contexts.append(context)
          questions.append(question)
          answers.append(answer)

  return contexts, questions, answers

In [352]:
train_contexts, train_questions, train_answers = read_data('/kaggle/input/rag-data/RAG_Datasets/BertTrainableDataset.json')
valid_contexts, valid_questions, valid_answers = read_data('/kaggle/input/rag-data/RAG_Datasets/BertTrainableDataset.json')

In [353]:
# print a random question and answer
print(f'There are {len(train_questions)} questions')
print(train_questions[1])
print(train_answers[1])

There are 282 questions
What was the initial role of AI in cybersecurity?
{'text': 'behavioral analytics to detect anomalies like unusual logins or data transfers', 'answer_start': 69}


In [354]:
def add_end_idx(answers, contexts):
  for answer, context in zip(answers, contexts):
    gold_text = answer['text']
    start_idx = answer['answer_start']
    end_idx = start_idx + len(gold_text)

    # sometimes squad answers are off by a character or two so we fix this
    if context[start_idx:end_idx] == gold_text:
      answer['answer_end'] = end_idx
    elif context[start_idx-1:end_idx-1] == gold_text:
      answer['answer_start'] = start_idx - 1
      answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
    elif context[start_idx-2:end_idx-2] == gold_text:
      answer['answer_start'] = start_idx - 2
      answer['answer_end'] = end_idx - 2 
    else:
      answer['answer_start']=start_idx
      answer['answer_end']=end_idx# When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(valid_answers, valid_contexts)

In [355]:
# You can see that now we get the answer_end also
print(train_questions[281])
print(train_answers[281])


How does the Alert AutoTriage feature help security teams?
{'text': 'It helps teams focus on real threats and reduces alert fatigue', 'answer_start': 8, 'answer_end': 70}


In [7]:
from transformers import BertTokenizerFast

model_path = '/kaggle/input/finetuned-llm-bert' #load the saved BERT Model for finetuning
# model = BertForQuestionAnswering.from_pretrained(model_path)
tokenizer = BertTokenizerFast.from_pretrained(model_path)

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
valid_encodings = tokenizer(valid_contexts, valid_questions, truncation=True, padding=True)

NameError: name 'train_contexts' is not defined

In [357]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [358]:
no_of_encodings = len(train_encodings['input_ids'])
print(f'We have {no_of_encodings} context-question pairs')

We have 282 context-question pairs


In [359]:
train_encodings['input_ids'][0]

[101,
 9932,
 2038,
 2042,
 17903,
 16941,
 3366,
 10841,
 15780,
 2144,
 2262,
 1010,
 3322,
 2083,
 14260,
 25095,
 2000,
 11487,
 2019,
 9626,
 11983,
 2066,
 5866,
 8833,
 7076,
 2030,
 2951,
 15210,
 1012,
 2058,
 2051,
 1010,
 2049,
 2535,
 4423,
 2000,
 3935,
 3698,
 4083,
 1998,
 2784,
 4083,
 1010,
 12067,
 16014,
 3512,
 5081,
 10788,
 1998,
 12978,
 3639,
 10595,
 1012,
 2651,
 1010,
 2058,
 3770,
 1003,
 1997,
 16941,
 3366,
 10841,
 15780,
 17088,
 13265,
 9932,
 1010,
 15929,
 16594,
 5906,
 2066,
 11416,
 6024,
 9932,
 2000,
 2994,
 3805,
 1997,
 2019,
 2412,
 1011,
 20607,
 5081,
 5957,
 1012,
 102,
 2043,
 2106,
 9932,
 4088,
 17903,
 16941,
 3366,
 10841,
 15780,
 1029,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [360]:
tokenizer.decode(train_encodings['input_ids'][190])

'[CLS] cybersecurity is crucial for protecting computer systems and sensitive information from threats like theft and fraud. protective measures for hardware include serial numbers, locks, and alarms. protecting information involves tactics like tracking user access, assigning passwords, and encrypting data. password tracking systems log user activities, and encryption ensures data can only be read by authorized users with the correct decryption key. [SEP] what are common protective measures for computer hardware? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [361]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    
    for i in range(len(answers)):
        try:
            answer_start = answers[i]['answer_start']
            answer_end = answers[i]['answer_end']

            # Correct the usage of char_to_token by passing context and position
            start_token = encodings.char_to_token(i, abs(answer_start))
            end_token = encodings.char_to_token(i, abs(answer_end - 1))  # Exclusive end

            # Handle cases where the tokenization has truncated the answer
            if start_token is None:
                start_token = tokenizer.model_max_length  # Fallback
            if end_token is None:
                end_token = tokenizer.model_max_length  # Fallback

            start_positions.append(start_token)
            end_positions.append(end_token)

        except Exception as e:
            print(f"Error processing answer {i}: {e}")
            continue

    # After processing all answers, update the encodings
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

    print(f"Total start positions: {len(start_positions)}")
    print(f"Total end positions: {len(end_positions)}")

add_token_positions(train_encodings, train_answers)
add_token_positions(valid_encodings, valid_answers)

Total start positions: 282
Total end positions: 282
Total start positions: 282
Total end positions: 282


In [362]:
train_encodings['start_positions'][:10]

[1, 14, 22, 16, 36, 29, 79, 71, 46, 18]

In [363]:
import torch
from torch.utils.data import Dataset

class SQuAD_Dataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        # Ensure 'start_positions' and 'end_positions' exist
        item = {
            "input_ids": torch.tensor(self.encodings["input_ids"][idx]),
            "attention_mask": torch.tensor(self.encodings["attention_mask"][idx]),
        }

        if "start_positions" in self.encodings and "end_positions" in self.encodings:
            item["start_positions"] = torch.tensor(self.encodings["start_positions"][idx])
            item["end_positions"] = torch.tensor(self.encodings["end_positions"][idx])

        return item


In [364]:
train_dataset = SQuAD_Dataset(train_encodings)
valid_dataset = SQuAD_Dataset(valid_encodings)

In [365]:
from torch.utils.data import DataLoader

# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16)

In [366]:
from transformers import BertForQuestionAnswering

model_path = '/kaggle/input/finetuned-llm-bert'
model = BertForQuestionAnswering.from_pretrained(model_path)


In [367]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

Working on cuda


In [368]:
print(len(train_dataset.encodings["input_ids"])),print(len(train_dataset.encodings["attention_mask"]))

282
282


(None, None)

In [369]:
from transformers import AdamW

N_EPOCHS = 5
optim = AdamW(model.parameters(), lr=5e-5)

model.to(device)
model.train()

for epoch in range(N_EPOCHS):
  loop = tqdm(train_loader, leave=True)
  for batch in loop:
    optim.zero_grad()

      
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    loss.backward()
    optim.step()

    loop.set_description(f'Epoch {epoch+1}')
    loop.set_postfix(loss=loss.item())
# model.save_pretrained(f"/epoch{epoch}")

Epoch 5: 100%|██████████| 18/18 [00:04<00:00,  3.64it/s, loss=1.24] 


In [648]:
model_path = '/kaggle/working/BERT-SQuAD'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('/kaggle/working/BERT-SQuAD/tokenizer_config.json',
 '/kaggle/working/BERT-SQuAD/special_tokens_map.json',
 '/kaggle/working/BERT-SQuAD/vocab.txt',
 '/kaggle/working/BERT-SQuAD/added_tokens.json',
 '/kaggle/working/BERT-SQuAD/tokenizer.json')

In [10]:
from transformers import BertForQuestionAnswering, BertTokenizerFast

model_path = '/kaggle/input/re-finetuned-bert/RE_Finetuned_BERT'
model = BertForQuestionAnswering.from_pretrained(model_path)
tokenizer = BertTokenizerFast.from_pretrained(model_path)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')
model = model.to(device)

Working on cuda


In [11]:
model

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [12]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.5 MB/s eta 0:00:00


In [13]:
from sentence_transformers import SentenceTransformer, util
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load a pre-trained model and move it to the GPU (if available)
transformer_model = SentenceTransformer('all-MiniLM-L6-v2').to(device) 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
transformer_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [15]:
def get_prediction(context, question):
  inputs = tokenizer.encode_plus(question, context, return_tensors='pt').to(device)
  outputs = model(**inputs)
  
  answer_start = torch.argmax(outputs[0])  
  answer_end = torch.argmax(outputs[1]) + 1 
  
  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
  
  return answer

def normalize_text(s):
  """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
  import string, re
  def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)
  def white_space_fix(text):
    return " ".join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()

  def remove_extra_tokens(text):
    return text.replace("[CLS]", "").replace("[SEP]", "").strip()

  return remove_extra_tokens(white_space_fix(remove_articles(remove_punc(lower(s)))))

def exact_match(prediction, truth):
    return bool(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()
  
  # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)
  
  common_tokens = set(pred_tokens) & set(truth_tokens)
  
  # if there are no common tokens then f1 = 0
  if len(common_tokens) == 0:
    return 0
  
  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)
  
  return round(2 * (prec * rec) / (prec + rec), 2)

def FindScore(s1,s2,Sentence_transformer):
    vec1=Sentence_transformer.encode(s1,convert_to_tensor=True)
    vec2=Sentence_transformer.encode(s2,convert_to_tensor=True)
    cosine_sim = util.pytorch_cos_sim(vec1, vec2)

    return cosine_sim
  
def question_answer(context, question,answer,Sentence_transformer):
  prediction = get_prediction(context,question)
  # em_score = exact_match(prediction, answer)
  # f1_score = compute_f1(prediction, answer)
  # print(f'Question: {question}')
  # print(f'Prediction: {prediction}')
  # print(f'True Answer: {answer}')
  # print(f'Exact match: {em_score}')
  # print(f'F1 score: {f1_score}\n')
  prediction.strip()
  if(len(prediction)==0 ):
      return [prediction,0]

  similarity_score = FindScore(answer,prediction,Sentence_transformer)

  return [prediction,similarity_score.item()]

In [496]:
## ON OUR DATASET 
contexts = ["The Earth orbits the Sun once every 365.25 days, which constitutes a year."]

questions = ['How long does it take for the Earth to orbit the Sun?']

answers = ["365"]

for question, answer in zip(questions, answers):
  print(question_answer(contexts[0], question, answer,transformer_model))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['365', tensor([[1.0000]], device='cuda:0')]


In [551]:
context = "Pradyumna Prahas is studying in Keshav Memorial Institute Of Technology he is 19 years old"

questions = ["How old is Pradyumna Prahas"]

answers = ["19"]

for question, answer in zip(questions, answers):
  print(question_answer(context, question, answer,transformer_model))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['19', tensor([[1.0000]], device='cuda:0')]


In [552]:
context = """Athens is the capital and largest city of Greece. Athens dominates the Attica region and is one of the world's oldest cities, 
             with its recorded history spanning over 3,400 years and its earliest human presence starting somewhere between the 11th and 7th millennium BC.
             Classical Athens was a powerful city-state. It was a center for the arts, learning and philosophy, and the home of Plato's Academy and Aristotle's Lyceum.
             It is widely referred to as the cradle of Western civilization and the birthplace of democracy, largely because of its cultural and political impact on the European continent—particularly Ancient Rome.
             In modern times, Athens is a large cosmopolitan metropolis and central to economic, financial, industrial, maritime, political and cultural life in Greece. 
             In 2021, Athens' urban area hosted more than three and a half million people, which is around 35% of the entire population of Greece.
             Athens is a Beta global city according to the Globalization and World Cities Research Network, and is one of the biggest economic centers in Southeastern Europe. 
             It also has a large financial sector, and its port Piraeus is both the largest passenger port in Europe, and the second largest in the world."""

questions = ["Which is the largest city in Greece?",
             "For what was the Athens center?",
             "Which city was the home of Plato's Academy?"]

answers = ["Athens", "center for the arts, learning and philosophy", "Athens"]

for question, answer in zip(questions, answers):
  print(question_answer(context, question, answer,transformer_model))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['athens', tensor([[1.0000]], device='cuda:0')]
['athens is the capital and largest city of greece. athens', [[0]]]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['athens', tensor([[1.0000]], device='cuda:0')]


In [553]:
context = """Angelos Poulis was born on 8 April 2001 in Nicosia, Cyprus. He is half Cypriot and half Greek. 
            He is currently studying at the Department of Informatics and Telecommunications of the University of Athens in Greece. 
            His scientific interests are in the broad field of Artificial Intelligence and he loves to train neural networks! 
            Okay, I'm Angelos and I'll stop talking about me right now."""

questions = ["When did Angelos born?",
             "In what university is Angelos studying now?",
             "What is Angelos' nationality?",
             "What are his scientific interests?",
             "What I will do right now?"]

answers = ["8 April 2001", "University of Athens", 
           "half Cypriot and half Greek", "Artificial Intelligence", 
           "stop talking about me"]

for question, answer in zip(questions, answers):
  print(question_answer(context, question, answer,transformer_model))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['8 april 2001 in nicosia, cyprus', tensor([[0.8871]], device='cuda:0')]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['university of athens', tensor([[1.]], device='cuda:0')]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['half cypriot and half greek', tensor([[1.]], device='cuda:0')]
['', [[0]]]
['half cypriot and half greek. he is currently studying at the department of informatics and telecommunications of the university', [[0]]]


### Testing the Accuracy Of the Model 

In [394]:
with open('/kaggle/input/rag-data/RAG_Datasets/BertTrainableDataset.json', 'rb') as f:
  test_data = json.load(f)

In [486]:
CONTEXTS=[]
QUESTIONS=[]
ANSWERS=[]

DATA=test_data["data"][0]["paragraphs"]
for i in range(len(DATA)):
    context=DATA[i]['context']
    for j in range(len(DATA[i]['qas'])):
        que=DATA[i]['qas'][j]['question']
        ans=DATA[i]['qas'][j]['answers'][0]['text']
        CONTEXTS.append(context)
        QUESTIONS.append(que)
        ANSWERS.append(ans)
len(CONTEXTS),len(QUESTIONS),len(ANSWERS)

(282, 282, 282)

In [615]:
ACCURACY_LIST=[]
predicted_answer=[]
for i in range(len(CONTEXTS)):
    # print(CONTEXTS[0],QUESTIONS[0],ANSWERS[0])
    l=question_answer(CONTEXTS[i], QUESTIONS[i], ANSWERS[i],transformer_model)
    ACCURACY_LIST.append(l[1])
    predicted_answer.append(l[0])
print(len(ACCURACY_LIST))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

282


In [616]:
print(len(ACCURACY_LIST),len(predicted_answer))

282 282


In [617]:
print(ACCURACY_LIST)

[0.32286515831947327, 0.9856256246566772, 0.42967575788497925, 0.07053522765636444, 0.5106699466705322, 0, 0, 0.33193933963775635, 0.5133306980133057, 0.434889018535614, 0.04174438863992691, 0, 0.13884885609149933, 0.9708271026611328, 0, 0, 0.4448700547218323, 0.47576117515563965, 0, 0.2496790885925293, 0.9125975370407104, 0.08551133424043655, 0.9503485560417175, 1.0, 0, 1.0, 0, 1.0, 1.0, 0, 0.2538270056247711, 0, 0.4156840443611145, 1.0, 0, 0.10784552991390228, 1.0, 0.39709049463272095, 0.35026708245277405, 0, 0.44875138998031616, 0.1709972321987152, 1.0, 0, 0.5509944558143616, 0.3916049301624298, 0.6428467631340027, 0.42971107363700867, 0.11012759059667587, 0.11012759059667587, 0.9618064761161804, 0.3980514407157898, 0.6459373235702515, 0.3444669842720032, 1.0, 0.30108842253685, 1.0, 0.9999999403953552, 0.01614663004875183, 0.633392333984375, 1.0, 0.5857731103897095, -0.052865251898765564, 1.0, 0.29623931646347046, 0.2765920162200928, 0, 0.6742863655090332, 0.1690530776977539, 0.4428

In [638]:
def eval_accuracy(l):
    #remove not predicted answers
    sum=0
    for i in range(len(l)):
        sum+=l[i]
    return sum/len(l)
    

In [640]:
# print("Accuracy after finetuning:- ",100*eval_accuracy(ACCURACY_LIST))
eval_accuracy(ACCURACY_LIST)

0.6830755057930946

In [ ]:
Accuracy Of the model on trained 280 questions is 68.30%

In [644]:
def show_data(idx):
    s="\n***************************\n"
    return "Context:\n"+CONTEXTS[idx]+s+"Question:\n"+QUESTIONS[idx]+s+"Actual Answer:\n"+ANSWERS[idx]+s+"Predicted Answer:\n"+predicted_answer[idx]+s

In [647]:
idx=int(input("Enter idx between 0-281 to see the predicted and actual answer of question with that id:-"))

print(show_data(idx))

Enter idx between 0-281 to see the predicted and actual answer of question with that id:- 281


Context:
The company also announced innovations like Singularity Hyperautomation, a no-code automation solution designed to streamline SOC workflows and reduce response times, particularly for complex threats such as ransomware. This solution integrates into the security analyst workflow, automatically generating playbooks based on peer-driven insights, making responses faster and more efficient. Another notable development is the Purple AI-powered Alert AutoTriage feature, which uses AI to assess and prioritize alerts, helping teams focus on real threats and reduce alert fatigue.
***************************
Question:
How does the Alert AutoTriage feature help security teams?
***************************
Actual Answer:
It helps teams focus on real threats and reduces alert fatigue
***************************
Predicted Answer:
singularity hyperautomation, a no - code automation solution designed to streamline soc workflows and reduce response times, particularly for complex threats such 